<a href="https://colab.research.google.com/github/cagBRT/Intro-to-Pandas/blob/master/Pandas_Performance_Enhancement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/Intro-to-Pandas.git cloned-repo
%cd cloned-repo
!ls

1. Download the dataset here:

In [ ]:
#https://www.kaggle.com/competitions/tabular-playground-series-sep-2021/rules

2. It will be be in zip format. <br>
Unzip the folder

3. Upload using file upload:<br>
>train.csv<br>
>test.csv


4. Wait for the upload to finish. <br>
This will take about 15 minutes

In [ ]:
import pandas as pd

tps = pd.read_csv("/content/train.csv")

In [ ]:
print(f"Shape of the tps dataset: {tps.shape}")

# Indexing efficiently
The fastest way of selecting rows and columns.

As you know, there are two indexing operators —
loc and iloc in Pandas.

Even though their difference won't matter much for small datasets, they will be pretty noticeable as data size increases.

Firstly, **for choosing a row or multiple rows, iloc is faster.**



## for choosing a row or multiple rows, iloc is faster.

In [ ]:
tps.iloc[range(10000)]

## loc is best for choosing columns with their labels

In [ ]:
tps.loc[:, ["f1", "f2", "f3"]]

## For sampling columns or rows, the built-in sample function is the fastest.

You may think that sampling using random indices with NumPy would be faster, but that’s not the case:

In [ ]:
#Sample 7 rows
tps.sample(7, axis=0)

In [ ]:
#Sample 5 columns and  7 rows
tps.sample(5, axis=1).sample(7, axis=0)

Use replace to replace specific values

While speed is the first benefit of replace, the second is its flexibility.

We can replace all question marks with NaN - an operation that would take multiple calls with index-based replacement.

In [ ]:
adult_income = pd.read_csv("adult.csv")

In [ ]:
adult_income.shape

In [ ]:
adult_income.columns

In [ ]:
adult_income.isna().sum()

In [ ]:
import numpy as np

adult_income.replace(to_replace="?", value=np.nan, inplace=True)

Also, replace allows using lists or dictionaries to change multiple values simultaneously:

In [ ]:
adult_income.isna().sum()

replace allows using lists or dictionaries to change multiple values simultaneously:



In [ ]:
adult_income.gender.value_counts()

In [ ]:
adult_income.replace(["Male", "Female"], ["M", "F"], inplace=True)

In [ ]:
adult_income.gender.value_counts()

When replacing a list of values with another, they will have a one-to-one, index-to-index mapping.

In [ ]:
adult_income["native-country"].value_counts()

In [ ]:
adult_income.replace({"United States": "USA", "US": "USA"}, inplace=True)

In [ ]:
adult_income["native-country"].value_counts()

In [ ]:
adult_income.education.value_counts()

In [ ]:
adult_income.income.value_counts()

In [ ]:
adult_income.replace(
    {
        "education": {"HS-grad": "High school", "Some-college": "College", "12th":"High school"},
        "income": {"<=50K": 0, ">50K": 1},
    },
    inplace=True,
)

In [ ]:
adult_income.education.value_counts()

In [ ]:
adult_income.income.value_counts()